# Installing faker lib

In [73]:
%pip install faker

Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'C:\Users\Eder\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.9_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip' command.


In [6]:
from faker import Faker
from faker.providers import BaseProvider
from datetime import datetime
import pandas as pd
import random
import json

fake = Faker()

# Generating fake `event_id`: random UUIDs 

In [14]:
Faker.seed(4321)
fake_event_id = fake.uuid4()
print(fake_event_id)

cc733c92-6853-45f6-8e49-bec741188ebb


# Generating fake `timestamp`: random timestamps with values until 3 years ago

In [39]:
fake_timestamp = datetime.strftime(fake.date_time_between(start_date='-3y', end_date='now'),"%Y-%m-%dT%H:%M:%S")
print(fake_timestamp)

2022-07-12T18:10:16


# Generating fake `domain`: random values based on valid grade names list

In [64]:
class ProjectDomainProvider(BaseProvider):
    def project_domain_name(self):
        list_project_domain_names = ['account','transaction']
        return random.choice(list_project_domain_names)

fake.add_provider(ProjectDomainProvider)

fake_project_domain_name = fake.project_domain_name()
print(fake_project_domain_name)

account


# Generating fake `status`: random values based on list

In [61]:
class StatusTypeProvider(BaseProvider):
    def status_type(self):
        list_status_types = ['ACTIVE','INACTIVE','SUSPENDED','BLOCKED', 'DELETED']
        return random.choice(list_status_types)

fake.add_provider(StatusTypeProvider)

fake_status_type = fake.status_type()
print(fake_status_type)

INACTIVE


# Generating fake `data`: random dict based on list

In [72]:
import random
 
# Initialize dictionary
test_dict = {'Gfg' : 1, 'is' : 2, 'best' : 3}
  
# printing original dictionary
print("The original dictionary is : ", test_dict)
  
# Method 3: Using random.sample()
# Get random dictionary pair in dictionary
# Using random.sample()
res = random.sample(test_dict.items(), 3)
  
# printing result
print("The random pair is : ", res)
#This code is contributed by Edula Vinay Kumar Reddy

The original dictionary is :  {'Gfg': 1, 'is': 2, 'best': 3}
The random pair is :  [('is', 2), ('best', 3), ('Gfg', 1)]


C:\Users\Eder\AppData\Local\Temp\ipykernel_2400\2300216194.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  res = random.sample(test_dict.items(), 3)


# Writing file with fake values

In [68]:
from json import dumps
from faker import Faker
import collections

database = []
filename = 'fake_events'
length = 10

for x in range(length):
    database.append(collections.OrderedDict([
        ('event_id', fake.uuid4()),
        ('timestamp', datetime.strftime(fake.date_time_between(start_date='-3y', end_date='now'),"%Y-%m-%dT%H:%M:%S")),
        ('domain', fake.project_domain_name()),
        ('event_type', "status-change"),
        ('data', collections.OrderedDict([
            ('id', fake.random_number(digits=6)),
            ('old_status', fake.status_type()),
            ('new_status', fake.status_type()),
            ('reason', fake.sentence(nb_words=5))
        ]))
    ]))

with open('../data/input/%s.json' % filename, 'w') as output:
    # turns date_between into string, circumvent json serialization
    output.write(dumps(database, indent=4, sort_keys=False, default=str))
print("Done.")

Done.


In [69]:
fake_data = pd.read_json("../data/input/fake_events.json")
fake_data

,event_id,timestamp,domain,event_type,data
0,ed3ea95a-27a7-4cc2-930a-4ef3755c26e0,2022-02-13 08:24:55,account,status-change,"{'id': 391724, 'old_status': 'SUSPENDED', 'new..."
1,18ef6b8f-b11b-482a-b9d8-e8349c1879f1,2022-12-03 15:53:05,account,status-change,"{'id': 996768, 'old_status': 'BLOCKED', 'new_s..."
2,c0f0edad-0d0e-4fba-b87b-ff8c91f86231,2021-08-31 03:10:03,account,status-change,"{'id': 131531, 'old_status': 'SUSPENDED', 'new..."
3,8fc61235-29b3-4d07-af91-07b976424bd2,2023-05-03 16:24:01,transaction,status-change,"{'id': 96451, 'old_status': 'INACTIVE', 'new_s..."
4,2fac4e12-d3e9-4f77-add0-b94530bde8ed,2021-11-11 05:34:41,transaction,status-change,"{'id': 263209, 'old_status': 'ACTIVE', 'new_st..."
5,18ff76eb-dbb2-4c70-93fd-998b8bebd7bc,2022-09-20 02:36:10,account,status-change,"{'id': 653657, 'old_status': 'BLOCKED', 'new_s..."
6,b253ca06-e16c-463a-b6e4-fa596793571d,2023-07-03 14:32:05,transaction,status-change,"{'id': 264737, 'old_status': 'BLOCKED', 'new_s..."
7,6d9263fc-54b0-4dfb-ac1a-c38186492c16,2022-01-25 02:37:51,account,status-change,"{'id': 563936, 'old_status': 'BLOCKED', 'new_s..."
8,60e31da4-abeb-40f6-a566-35909d31cf83,2022-02-27 01:38:27,transaction,status-change,"{'id': 86233, 'old_status': 'BLOCKED', 'new_st..."
9,93acd235-0376-41b2-b5a2-d1dfe8e8953c,2020-12-12 14:05:24,transaction,status-change,"{'id': 375147, 'old_status': 'INACTIVE', 'new_..."
